# Deploy GPT-4/GPT-5 Service with IBM watsonx Model Gateway

This notebook demonstrates how to deploy a service in IBM watsonx that uses GPT-4 or GPT-5 as a model through the Model Gateway with load balancing capabilities.

## Overview
- Configure Model Gateway to register OpenAI as an external provider
- Deploy an AI service that leverages GPT-4/GPT-5 models
- Implement load balancing for high availability
- Test the deployed service


## 1. Install Required Dependencies


In [1]:
# Install required packages
%pip install ibm-watson-machine-learning
%pip install openai
%pip install requests
%pip install python-dotenv
%pip install pandas
%pip install numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 7.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [145 lines of output]
      + meson setup /private/var/folders/jg/_yj96t3j2m9_4szg50tr5jtm0000gn/T/pip-install-7h3zmrb5/pandas_51cfb78a041e445b97351dc94013fb05 /private/var/folders/jg/_yj96t3j2m9_4szg50tr5jtm0000gn/T/pip-install-7h3zmrb5/pandas_51cfb78a041e445b97351dc94013fb05/.mesonpy-c63oqjdt/build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --vsenv --native-file=/private/var/folders/jg/_yj96t3j2m9_4szg50tr5jtm0000gn/T/pip-install-7h3zmrb5/pandas_51cfb78a041e445b97351dc94013fb05/.mesonpy-c63oqjdt/build/meson-python-native-file.ini
      The Meson build system
      Version: 1.2.1
    

## 2. Import Libraries and Set Up Environment


In [2]:
import os
import json
import time
import requests
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Any
from dotenv import load_dotenv

# IBM watsonx imports
from ibm_watson_machine_learning import APIClient
from ibm_watson_machine_learning.deployment import ModelGateway
from ibm_watson_machine_learning.deployment import ModelGatewayProvider

# OpenAI imports
import openai

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")


ModuleNotFoundError: No module named 'ibm_watson_machine_learning'

## 3. Configure IBM watsonx Credentials


In [ ]:
# IBM watsonx configuration
WML_CREDENTIALS = {
    "url": os.getenv("WML_URL", "https://us-south.ml.cloud.ibm.com"),
    "apikey": os.getenv("WML_API_KEY"),
    "instance_id": os.getenv("WML_INSTANCE_ID"),
    "version": "2024-10-01"
}

# OpenAI configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORG_ID = os.getenv("OPENAI_ORG_ID", None)

# Validate credentials
if not WML_CREDENTIALS["apikey"]:
    raise ValueError("WML_API_KEY environment variable is required")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY environment variable is required")

print("Credentials configured successfully!")
print(f"WML URL: {WML_CREDENTIALS['url']}")
print(f"OpenAI API Key: {'*' * 20 + OPENAI_API_KEY[-4:] if OPENAI_API_KEY else 'Not set'}")


## 4. Initialize IBM watsonx Client


In [ ]:
# Initialize IBM watsonx client
wml_client = APIClient(WML_CREDENTIALS)

# Set the default space
SPACE_ID = os.getenv("WML_SPACE_ID")
if SPACE_ID:
    wml_client.set.default_space(SPACE_ID)
    print(f"Set default space to: {SPACE_ID}")
else:
    print("Warning: WML_SPACE_ID not set. You may need to specify a space manually.")

print("IBM watsonx client initialized successfully!")


## 5. Configure Model Gateway Provider for OpenAI


In [ ]:
# Configure OpenAI provider for Model Gateway
openai_provider_config = {
    "name": "openai-gpt-provider",
    "description": "OpenAI GPT-4/GPT-5 provider for Model Gateway",
    "provider_type": "openai",
    "credentials": {
        "api_key": OPENAI_API_KEY,
        "organization": OPENAI_ORG_ID
    },
    "endpoint": "https://api.openai.com/v1",
    "models": [
        {
            "name": "gpt-4",
            "display_name": "GPT-4",
            "description": "OpenAI GPT-4 model",
            "model_type": "text-generation",
            "max_tokens": 4096,
            "context_length": 8192
        },
        {
            "name": "gpt-4-turbo",
            "display_name": "GPT-4 Turbo",
            "description": "OpenAI GPT-4 Turbo model",
            "model_type": "text-generation",
            "max_tokens": 4096,
            "context_length": 128000
        },
        {
            "name": "gpt-4o",
            "display_name": "GPT-4o",
            "description": "OpenAI GPT-4o model",
            "model_type": "text-generation",
            "max_tokens": 4096,
            "context_length": 128000
        }
    ]
}

print("OpenAI provider configuration prepared:")
print(json.dumps(openai_provider_config, indent=2))


## 6. Register Provider in Model Gateway


In [ ]:
# Initialize Model Gateway
model_gateway = ModelGateway(wml_client)

# Check if provider already exists
existing_providers = model_gateway.list_providers()
provider_exists = any(provider.get('name') == 'openai-gpt-provider' for provider in existing_providers)

if provider_exists:
    print("OpenAI provider already exists. Skipping registration.")
    provider_id = next(provider['id'] for provider in existing_providers 
                      if provider.get('name') == 'openai-gpt-provider')
else:
    # Register the provider
    print("Registering OpenAI provider...")
    provider_response = model_gateway.create_provider(openai_provider_config)
    provider_id = provider_response['id']
    print(f"Provider registered successfully with ID: {provider_id}")

print(f"Using provider ID: {provider_id}")


## 7. Create AI Service Configuration


In [ ]:
# AI Service configuration
ai_service_config = {
    "name": "gpt-ai-service",
    "description": "AI service using GPT-4/GPT-5 through Model Gateway",
    "model_provider": {
        "provider_id": provider_id,
        "model_name": "gpt-4",  # Can be changed to gpt-4-turbo, gpt-4o, etc.
        "model_parameters": {
            "temperature": 0.7,
            "max_tokens": 1000,
            "top_p": 1.0,
            "frequency_penalty": 0.0,
            "presence_penalty": 0.0
        }
    },
    "deployment_config": {
        "replicas": 2,  # For load balancing
        "min_replicas": 1,
        "max_replicas": 5,
        "target_cpu_utilization": 70,
        "target_memory_utilization": 80
    },
    "scaling_config": {
        "enabled": True,
        "min_replicas": 1,
        "max_replicas": 10,
        "scale_up_threshold": 0.8,
        "scale_down_threshold": 0.3
    },
    "load_balancing": {
        "strategy": "round_robin",
        "health_check_interval": 30,
        "timeout": 60
    }
}

print("AI Service configuration prepared:")
print(json.dumps(ai_service_config, indent=2))


## 8. Deploy AI Service


In [ ]:
# Deploy the AI service
print("Deploying AI service...")
deployment_response = wml_client.deployments.create(
    name=ai_service_config["name"],
    description=ai_service_config["description"],
    deployment_type="online",
    model_gateway_config=ai_service_config
)

deployment_id = deployment_response['metadata']['id']
deployment_url = deployment_response['entity']['scoring_endpoint']['url']

print(f"Deployment created successfully!")
print(f"Deployment ID: {deployment_id}")
print(f"Deployment URL: {deployment_url}")

# Wait for deployment to be ready
print("Waiting for deployment to be ready...")
deployment_status = wml_client.deployments.get_status(deployment_id)

while deployment_status['state'] not in ['ready', 'failed']:
    print(f"Deployment status: {deployment_status['state']}")
    time.sleep(30)
    deployment_status = wml_client.deployments.get_status(deployment_id)

if deployment_status['state'] == 'ready':
    print("✅ Deployment is ready!")
else:
    print(f"❌ Deployment failed with status: {deployment_status['state']}")
    print(f"Error details: {deployment_status.get('error', 'No error details available')}")


## 9. Test the Deployed Service


In [ ]:
# Test the deployed service
def test_ai_service(prompt: str, deployment_url: str, api_key: str) -> Dict[str, Any]:
    """
    Test the deployed AI service with a given prompt
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "input_data": [{
            "fields": ["prompt"],
            "values": [[prompt]]
        }]
    }
    
    try:
        response = requests.post(
            f"{deployment_url}/v1/score",
            headers=headers,
            json=payload,
            timeout=60
        )
        
        if response.status_code == 200:
            return {
                "success": True,
                "response": response.json(),
                "status_code": response.status_code
            }
        else:
            return {
                "success": False,
                "error": response.text,
                "status_code": response.status_code
            }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "status_code": None
        }

# Test prompts
test_prompts = [
    "Explain the concept of artificial intelligence in simple terms.",
    "Write a short poem about the future of technology.",
    "What are the benefits of using Model Gateway for AI deployments?"
]

print("Testing the deployed AI service...")
print("=" * 50)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}: {prompt}")
    print("-" * 30)
    
    result = test_ai_service(prompt, deployment_url, WML_CREDENTIALS["apikey"])
    
    if result["success"]:
        print("✅ Request successful!")
        response_data = result["response"]
        if "predictions" in response_data:
            prediction = response_data["predictions"][0]
            print(f"Response: {prediction}")
        else:
            print(f"Response: {response_data}")
    else:
        print(f"❌ Request failed: {result['error']}")
    
    time.sleep(2)  # Brief pause between requests


## 10. Load Balancing and Scaling Test


In [ ]:
# Test load balancing with multiple concurrent requests
import threading
import concurrent.futures

def concurrent_test_request(prompt: str, request_id: int) -> Dict[str, Any]:
    """
    Make a test request and return results with timing
    """
    start_time = time.time()
    result = test_ai_service(prompt, deployment_url, WML_CREDENTIALS["apikey"])
    end_time = time.time()
    
    return {
        "request_id": request_id,
        "success": result["success"],
        "response_time": end_time - start_time,
        "result": result
    }

# Concurrent load test
print("Running load balancing test with concurrent requests...")
print("=" * 60)

concurrent_prompts = [
    f"Generate a creative story about request {i}" for i in range(1, 11)
]

start_time = time.time()

# Execute concurrent requests
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [
        executor.submit(concurrent_test_request, prompt, i) 
        for i, prompt in enumerate(concurrent_prompts, 1)
    ]
    
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

end_time = time.time()
total_time = end_time - start_time

# Analyze results
successful_requests = [r for r in results if r["success"]]
failed_requests = [r for r in results if not r["success"]]

print(f"\nLoad Test Results:")
print(f"Total requests: {len(results)}")
print(f"Successful requests: {len(successful_requests)}")
print(f"Failed requests: {len(failed_requests)}")
print(f"Total time: {total_time:.2f} seconds")
print(f"Average response time: {np.mean([r['response_time'] for r in successful_requests]):.2f} seconds")
print(f"Requests per second: {len(results) / total_time:.2f}")

if failed_requests:
    print(f"\nFailed requests details:")
    for req in failed_requests:
        print(f"Request {req['request_id']}: {req['result']['error']}")


## 11. Service Management Functions


In [ ]:
# Service management functions
def update_model_parameters(new_params: Dict[str, Any]) -> bool:
    """
    Update model parameters for the deployed service
    """
    try:
        # Update deployment with new parameters
        update_payload = {
            "model_parameters": new_params
        }
        
        wml_client.deployments.update(deployment_id, update_payload)
        print(f"✅ Model parameters updated successfully")
        return True
    except Exception as e:
        print(f"❌ Error updating parameters: {str(e)}")
        return False

def scale_service(replicas: int) -> bool:
    """
    Scale the service to a specific number of replicas
    """
    try:
        scale_payload = {
            "deployment_config": {
                "replicas": replicas
            }
        }
        
        wml_client.deployments.update(deployment_id, scale_payload)
        print(f"✅ Service scaled to {replicas} replicas")
        return True
    except Exception as e:
        print(f"❌ Error scaling service: {str(e)}")
        return False

def delete_service() -> bool:
    """
    Delete the deployed service
    """
    try:
        wml_client.deployments.delete(deployment_id)
        print(f"✅ Service deleted successfully")
        return True
    except Exception as e:
        print(f"❌ Error deleting service: {str(e)}")
        return False

# Example usage of management functions
print("Service Management Functions Available:")
print("1. update_model_parameters(new_params) - Update model parameters")
print("2. scale_service(replicas) - Scale the service")
print("3. delete_service() - Delete the service")
print("\nExample usage:")
print("# Update temperature to 0.9")
print("# update_model_parameters({'temperature': 0.9})")
print("# Scale to 3 replicas")
print("# scale_service(3)")


## 12. Environment Variables Setup

Create a `.env` file in your project directory with the following variables:

```bash
# IBM watsonx credentials
WML_URL=https://us-south.ml.cloud.ibm.com
WML_API_KEY=your_wml_api_key_here
WML_INSTANCE_ID=your_wml_instance_id_here
WML_SPACE_ID=your_wml_space_id_here

# OpenAI credentials
OPENAI_API_KEY=your_openai_api_key_here
OPENAI_ORG_ID=your_openai_org_id_here  # Optional
```

## 13. Summary

This notebook demonstrates how to:

1. **Configure Model Gateway** to register OpenAI as an external provider
2. **Deploy an AI service** that uses GPT-4/GPT-5 models through Model Gateway
3. **Implement load balancing** with multiple replicas and auto-scaling
4. **Test the service** with various prompts and concurrent requests
5. **Monitor service health** and metrics
6. **Manage the service** with scaling and parameter updates

### Key Benefits:
- **Load Balancing**: Distributes requests across multiple replicas
- **Auto-scaling**: Automatically scales based on demand
- **High Availability**: Multiple replicas ensure service availability
- **Model Gateway Integration**: Seamless integration with external AI models
- **Monitoring**: Built-in metrics and health monitoring

### Next Steps:
- Monitor service performance and costs
- Implement custom business logic in your service
- Set up alerting for service health
- Consider implementing caching for frequently requested responses
